<a href="https://colab.research.google.com/github/Abdxul/Abdxul/blob/main/Generateur_De_Texte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os


In [ ]:
path_to_file = './shakespeare.txt'

In [ ]:
text = open(path_to_file,'rb').read()
text = text.decode(encoding='utf-8')
print('Total number of caracters in the corpus is:',len(text))
print('The first 100 caracters of the corpus are as follow:\n',text[:100])


Total number of caracters in the corpus is: 1115394
The first 100 caracters of the corpus are as follow:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
vocab = sorted(set(text))
print('The number of unique caracters in the corpus is',len(vocab))
print('A slice of the unique caracters set:\n',vocab[:10])


The number of unique caracters in the corpus is 65
A slice of the unique caracters set:
 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100

sequences = char_dataset.batch(seq_length+1,drop_remainder = True)


In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)

  ])
  return model

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size =BATCH_SIZE
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss =loss)

In [ ]:
checkpoint_dir='./training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
EPOCHS = 30
history = model.fit(dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])


Epoch 1/30
172/172 [==============================] - 16s 60ms/step - loss: 2.6925
Epoch 2/30
172/172 [==============================] - 12s 58ms/step - loss: 1.9635
Epoch 3/30
172/172 [==============================] - 12s 58ms/step - loss: 1.6993
Epoch 4/30
172/172 [==============================] - 12s 60ms/step - loss: 1.5499
Epoch 5/30
172/172 [==============================] - 12s 61ms/step - loss: 1.4604
Epoch 6/30
172/172 [==============================] - 13s 62ms/step - loss: 1.4008
Epoch 7/30
172/172 [==============================] - 12s 61ms/step - loss: 1.3543
Epoch 8/30
172/172 [==============================] - 12s 61ms/step - loss: 1.3145
Epoch 9/30
172/172 [==============================] - 13s 61ms/step - loss: 1.2803
Epoch 10/30
172/172 [==============================] - 12s 61ms/step - loss: 1.2477
Epoch 11/30
172/172 [==============================] - 12s 62ms/step - loss: 1.2164
Epoch 12/30
172/172 [==============================] - 12s 62ms/step - loss: 1.1849
E

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_3 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_3 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:


def generate_text(model, num_generate, temperature, start_string):
    # Converting the start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Reset the initial state of the model
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


In [ ]:
generated_text = generate_text(model,
                                num_generate = 1000,
                                temperature=1,
                                start_string=u"ROMEO")
print(generated_text)

ROMEO:
Practisfire the course of joy, I cheque'd my friends,
I' the curses in heaven and earth,
My tongue say so? then I'll set again; would bring
Themorter: therefore it is faults I mean,
When Richmond strew,
Enchrise to be absolved!

DUKE VINCENTIO:
Should say this it is so pale edry the costard?

CURTIS:
But you are certainly a gentle maid hath kill'd
Commit me with the glasses he
With old opposite lives; then girls one that are like the bottommon
my daughter and his prattle still
As or bathem bold: 'tis true, I give thee,
And so I take my letter breath eyes,
My mirth it is become a prophecy by his son.
For your looks are gentle fill.

KING RICHARD III:
You will find upright love,
Which these blacks in me for justice when he wakes,
To be the other in this captain I shell understand,
Is grows the dukedom since, to Corioli have
hand for an edge hours have paphed judgment
That I have following his son fright to be drunk,
That you shall goes with Angelo.

I may mine,
If he shall smile i